<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/ML/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://a.teleboss.ru/play/c851b57a-ccd5-408c-9484-e3d00c79b46e

https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system/data?select=movies.csv

# Загрузка датасета с kaggle

In [42]:
!curl -L -o movie-recommendation-system.zip https://www.kaggle.com/api/v1/datasets/download/parasharmanas/movie-recommendation-system
!unzip -o /content/movie-recommendation-system.zip
!rm *.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  164M  100  164M    0     0   113M      0  0:00:01  0:00:01 --:--:-- 88.7M
Archive:  /content/movie-recommendation-system.zip
  inflating: movies.csv              
  inflating: ratings.csv             


In [43]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100

In [44]:
ratings_full_df = pd.read_csv("/content/ratings.csv", nrows=200_000)
movies_df = pd.read_csv("/content/movies.csv")

In [45]:
ratings_full_df.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [46]:
ratings_full_df.rating.pipe(np.sum)

713282.0

In [47]:
movies_df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [58]:
X_ = ratings_full_df.drop("rating", axis=1)
y_ = ratings_full_df['rating']

# Подготовка pipline для обработки
## Часть 1 - Пишем cамый примитивный Estimator

Он будет использоваться как baseline для дальнейших экспериментов

In [51]:
class NaivePredictor(ClassifierMixin, BaseEstimator):

    def __init__(self, a=0.5, avg=np.median):
        self.rating_movie_mean_ = None  # Для хранения средней оценки фильма
        self.rating_user_mean_ = None  # Для хранения средней оценки которую ставит пользователь
        self.a = a  # Параметр, который будем менять при подборе по сетке GridSearchCV для определения долей влияния средней оценки фильма и среднй оценки пользователя
        self.avg = avg # Параметр, который будем менять при подборе по сетке GridSearchCV для выбора способа оценки среднего выборки
        self.classes_ = [0] # Технический момент, никак не используется, но атрибут должен существовать для корректной работы GridSearchCV

    def fit(self, X, y):
        concatenated_df = pd.concat([X, y], axis=1)
        self.rating_movie_mean_ = concatenated_df.groupby('movieId')['rating'].apply(self.avg) # Средняя оценка фильма
        self.rating_user_mean_ = concatenated_df.groupby('userId')['rating'].apply(self.avg) # Средняя оценка, которую ставит пользователь
        self.global_avg = y.pipe(self.avg) # Средняя всех оценок в тренировочном датасете

        return self

    def predict(self, X):
        """
        Предсказывает средний рейтинг фильма как сумму долей средней оценки фильма и среднй оценки, которую обычно ставит пользователь
        Если `movieId` нет в обучающей выборке, возвращает средний рейтинг по всем фильмам.
        """
        if self.rating_movie_mean_ is None:
            raise ValueError("Model is not fitted yet. Call `fit` before `predict`.")

        predictions = (X['movieId'].map(self.rating_movie_mean_) * self.a +
                       X['userId'].map(self.rating_user_mean_) * (1 - self.a))\
                       .fillna(self.global_avg) # Заполняем пропущенные значения средним рейтингом всех фильмов

        return predictions

In [52]:
pipe = Pipeline(
    [

        ("NaivePredictor", NaivePredictor())
    ]
)

In [53]:
splitter = KFold(
    n_splits=2,
    shuffle=True,
    random_state=42
)

In [54]:
param_grid = {
    "NaivePredictor__a": np.arange(start=0.1, stop=1.1, step=0.1),
    "NaivePredictor__avg": [np.mean, np.median]
}

In [55]:
search = GridSearchCV(pipe,
                      param_grid,
                      cv=splitter,
                      scoring='neg_mean_absolute_error',
                      n_jobs=-1,
                      verbose=3,
                      return_train_score=True,
                      error_score="raise")

search.fit(X_, y_)
print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
Best parameter (CV score=-0.70413):
{'NaivePredictor__a': 0.4, 'NaivePredictor__avg': <function median at 0x7bffe353caf0>}


In [56]:
def evaluate_model(y_true, y_predicted):
  result = pd.DataFrame.from_dict({
          "MAE": f'{mean_absolute_error(y_true, y_predicted):.3f}',
          "MSE": f'{mean_squared_error(y_true, y_predicted):.3f}',
          "MAPE": f'{mean_absolute_percentage_error(y_true, y_predicted):.2%}',
          "R2_score": f'{r2_score(y_true, y_predicted):.3f}'
  },
  orient='index', columns=['value'])
  return result

In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X_,
    y_,
    test_size=0.2, # <--- Доля теста к тотал данным
    random_state=42  # <--- Мешать ли данные или делить по индексам
)

In [156]:
evaluate_model(y_test, search.predict(X_test))

,value
MAE,0.666
MSE,0.792
MAPE,30.93%
R2_score,0.273


In [157]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        movies_data,

    ):

        self.mlb = MultiLabelBinarizer()
        self.movies_data = movies_data

    def fit(self, X, y):

        X_train_copy = X.copy()
        X_train_copy = pd.concat([X_train_copy, y], axis = 1)
        X_train_copy = X_train_copy.merge(movies_df[['movieId', 'genres']], on='movieId', how='left')

        self.rating_user_mean = X_train_copy.groupby("userId", as_index=False)\
                                               .rating.mean()\
                                               .rename({"rating": "rating_user_mean"}, axis=1)

        self.rating_movie_mean = X_train_copy.groupby("movieId", as_index=False)\
                                                .rating.mean()\
                                                .rename({"rating": "rating_movie_mean"}, axis=1)


        self.mlb.fit(self.movies_data.genres.apply(lambda x: x.split('|')))


        mlb_encoded_genres = pd.DataFrame(self.mlb.transform(X_train_copy.genres.apply(lambda x: x.split('|'))), \
                                          columns=self.mlb.classes_)

        temp = pd.concat([X_train_copy.userId, mlb_encoded_genres.mul(y, axis=0)], axis=1).groupby("userId").agg(lambda x: np.mean([_ for _ in x if _ > 0]))
        self.user_genres_rating = temp.T.fillna(temp.mean(axis=1)).T
        self.mean_rating = y.mean()
        return self

    def transform(self, X:pd.DataFrame):

        X_copy= X.copy()
        X_copy = X_copy.merge(self.rating_user_mean, how='left', on='userId') \
                       .merge(self.rating_movie_mean, how='left', on='movieId') \
                       .merge(movies_df[['movieId', 'genres']], on='movieId', how='left').fillna(2.5)

        datetime_ = pd.to_datetime(X_copy.timestamp, unit='s')

        X_copy['year'] = datetime_.dt.year
        X_copy['month'] = datetime_.dt.month
        X_copy['day_of_week'] = datetime_.dt.day_of_week
        X_copy['time'] = pd.cut(datetime_.dt.hour,
                                include_lowest=True,
                                bins=[0, 5, 12, 17, 21, 24],
                                right=False,
                                ordered=False,
                                labels=['night', 'morining', 'afternoon', 'evening', 'night'])

        X_copy['genres'] = X_copy.genres.apply(lambda x: x.split('|'))

        mlb_encoded_genres = pd.DataFrame(self.mlb.transform(X_copy.genres), \
                                         columns=self.mlb.classes_)

        #X_copy = pd.concat([X_copy, mlb_encoded_genres], axis=1)
        X_copy = pd.concat([X_copy, self.user_genres_rating.merge(X_copy.userId, how='right', right_on='userId', left_index=True).iloc[:,:-1].apply(lambda col: col.fillna(col.mean()), axis=0) * mlb_encoded_genres], axis=1)

        """
        X_copy = X_copy.merge(pd.concat([mlb_encoded_genres,\
                                X_copy.userId], axis = 1)
                               .groupby("userId", as_index = False).mean(), \
                     how='left',
                     on='userId', \
                     suffixes = ('', '_gen_mean'))
        """


        X_copy.drop(columns=['timestamp', 'genres'], inplace=True)
        return X_copy.set_index(X.index)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

    def inverse_transform(self):
        pass

In [49]:
onehot_columns = ['time'] # список колонок, которые будем кодировать c OneHotEncoder

In [50]:
col_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), onehot_columns),
    ],
    remainder='passthrough',          # Оставляем необрабатываемые колонки как есть, не удаляем их
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas


In [158]:
final_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("linear_regression", Ridge())
    ]
)


In [162]:
lasso_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Lasso())
    ]
)

ridge_pipe = Pipeline(
    [
        ("Custom_transformer", CustomTransformer(movies_data=movies_df)),
        ("OneHotEncoder", col_transformer),
        ("simple_model", Ridge())
    ]
)

In [165]:
param_grid = {
    "simple_model__alpha": np.logspace(-1, 3, 10),
    "simple_model__max_iter": [100, 1000]
}

In [166]:
%%time
for model in [ridge_pipe]:
    ### Передадим в GridSearchCV
    search = GridSearchCV(model, param_grid,
                          cv=splitter, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10)

    search.fit(X_, y_)

    print(f"Best parameter (CV score={search.best_score_:.5f}):")
    print(search.best_params_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Best parameter (CV score=-0.70248):
{'simple_model__alpha': 1000.0, 'simple_model__max_iter': 100}
CPU times: user 7.36 s, sys: 535 ms, total: 7.9 s
Wall time: 2min 13s


In [167]:
evaluate_model(y_test, search.predict(X_test))

,value
MAE,0.636
MSE,0.686
MAPE,27.18%
R2_score,0.370


In [173]:
search.best_estimator_.named_steps.simple_model.coef_

array([ 1.04059101e-02, -2.49558826e-03,  4.40923125e-02,  3.32142263e-06,
        2.91875474e-07,  7.95015684e-01,  8.42661449e-01, -6.23495386e-03,
        2.38087477e-04,  4.83806125e-03,  4.24583253e-02,  8.46547380e-03,
        5.09682247e-03,  1.65008812e-02,  1.22474934e-03,  1.33240856e-02,
        1.22784120e-02,  5.88045315e-02,  1.85343304e-02,  8.13458326e-03,
        2.26325030e-02,  3.19501603e-02,  3.82566361e-03,  1.24249082e-02,
        8.71857186e-03,  6.19804279e-03,  8.36666108e-03,  7.40376974e-03,
        1.32564675e-02,  3.00967059e-02])

In [ ]:
evaluate_model(y_test, final_pipe.fit(X_train, y_train).predict(X_test))

In [ ]:
from sklearn.model_selection import KFold

splitter = KFold(
    n_splits=2,
    shuffle=True,
    random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate

cv_result = cross_validate(pipe,
                           X_, y_,
                           scoring='neg_mean_absolute_error',
                           cv=splitter, return_train_score=True)

cv_result

In [ ]:
param_grid = {}
search = GridSearchCV(final_pipe, param_grid,
                      cv=splitter, scoring='neg_mean_absolute_error')

In [ ]:
search.fit(X_, y_)
print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

In [ ]:
search.best_score_

In [ ]:
list(splitter.split(X_, y_))

In [ ]:
movies_df.genres.isna().sum()

In [ ]:
CustomTransformer2(movies_data=movies_df).fit(X_train, y_train).transform(X_test)

In [ ]:
 CustomTransformer2(movies_data=movies_df).fit(X_train, y_train).transform(X_test).isna().sum().loc[lambda x: x > 0]